# CS-GY 6083 Principals of Database Systems - Spring 2024
* **Author**: [Nicola Maiorana]
* **Date**: [2024-03-23]
* **Email**: [nam10102@nyu.edu]
* **Class**: [CS-GY 6083]

## Introduction
This notebook will demonstrate all the features I developed for the project. It will present the database schema I used for the project and then the various capabilities required for the project.

## Architecture
This project will demonstrate a 2-tiered database architecture where the Python modules/classes will interact directly with the MySQL database. The name of the database is called "album_information" and is an example of a method to store record album data.

### Database
The database will be coded in the record_album_information.sql file. Here all the necessary constructs from tables to triggers will be defined. Also, sample data for the database will be included so that the database can be recreated with a single execution. The database will consist of the following:
#### Tables
- record_artists - Creators of the albums
- group_members - Band members to form the artist groups
- record_genres - The overall musical style of the album and each song on the album
- record_labels - The organizations responsible for producing the record albums
- record_tracks - The songs on each album
- record_sales - Various periodic sales (Yearly) for each album
- record_albums - The information all linked together to provide data about each record album
- members_to_artists - Link the band members to the artist records

#### Views
- album_information - High level album information
- album_information_details - Detailed information including songs
- band_members - Band members and the dates they were part of the group

### Python Tools
- Various Database tools to interact with a local server
    - mysql.connector: connectivity to the server
    - sqlalchemy: For construction of the Object Relational Models (ORMs)
    - Pandas to render the query data into easy to view structures
- Custom classes created for this project
-   db_utils.py - Basic interaction with the database and helpers for the ORMs





## Import required modules

In [1]:
import project.tools.db_utils as dbu
from sqlalchemy.sql import text

## Get metadata from the database using SQLAlchemy

### Table Names

In [2]:
for table_name in dbu.get_table_names():
    print(f'Table: {table_name}')

Table: group_members
Table: members_to_artists
Table: record_albums
Table: record_artists
Table: record_genres
Table: record_labels
Table: record_sales
Table: record_tracks


### View Names

In [3]:
for view_names in dbu.get_view_names():
    print(f'View: {view_names}')

View: album_information
View: album_information_details
View: band_members


### Column Names

In [4]:
table_name = 'members_to_artists'
for column in dbu.get_columns(table_name):
    print(f"Column ({table_name}): {column['name']:<25} Type: {column['type']}")

Column (members_to_artists): members_to_artists_id     Type: INTEGER
Column (members_to_artists): member_id                 Type: INTEGER
Column (members_to_artists): artist_id                 Type: INTEGER
Column (members_to_artists): member_from_date          Type: DATE
Column (members_to_artists): member_to_date            Type: DATE


### Simple Query

In [5]:
pandas_index_settings = ['name', 'release_date', 'artist_name', 'record_label_name', 'track_number']
display(dbu.sqlalchemy_query_to_df('select * from album_information_details order by release_date', pandas_index_settings))

genre_name  \
name                      release_date artist_name record_label_name       track_number              
The Dark Side of the Moon 1973-03-01   Pink Floyd  Harvest Capitol Records 1                  Rock   
                                                                           2                  Rock   
                                                                           3                  Rock   
                                                                           4                  Rock   
                                                                           5                  Rock   
                                                                           6                  Rock   
                                                                           7                  Rock   
                                                                           8                  Rock   
                                                                           9                  Rock   
                                                                           10                 Rock   
Boston                    1976-08-25   Boston      Epic Records            1                  Rock   
                                                                           2                  Rock   
                                                                           3                  Rock   
                                                                           4                  Rock   
                                                                           5                  Rock   
                                                                           6                  Rock   
                                                                           7                  Rock   
                                                                           8                  Rock   

                                                                                                           track_name  
name                      release_date artist_name record_label_name       track_number                                
The Dark Side of the Moon 1973-03-01   Pink Floyd  Harvest Capitol Records 1                              Speak to Me  
                                                                           2                     Breathe (In the Air)  
                                                                           3                               On the Run  
                                                                           4                                     Time  
                                                                           5                 The Great Gig in the Sky  
                                                                           6                                    Money  
                                                                           7                              Us and Them  
                                                                           8                      Any Colour You Like  
                                                                           9                             Brain Damage  
                                                                           10                                 Eclipse  
Boston                    1976-08-25   Boston      Epic Records            1                      More Than a Feeling  
                                                                           2                            Peace of Mind  
                                                                           3                       Foreplay/Long Time  
                                                                           4                         Rock & Roll Band  
                                                                           5                                   Smokin  
       

## Procedure DDL

### Procedure to count the number of sales for a given album
```
DROP PROCEDURE IF EXISTS count_record_sales;

DELIMITER //
CREATE PROCEDURE count_record_sales(IN album_id INT, OUT sales_count INT)
BEGIN
	SET @album_id = album_id;
    SELECT 
        COUNT(sale_id) INTO sales_count
    FROM RECORD_SALES
    WHERE RECORD_SALES.album_id = @album_id;

END //
DELIMITER ;
```


In [3]:
import project.tools.db_utils as dbu
from sqlalchemy.sql import text

with dbu.get_session() as session:
    album_id = 1
    print(f'Number of sales for album 1: {len(dbu.sqlalchemy_query_to_df("select * from record_sales where album_id = 1", "sale_id"))}')
    call_stored_procedure = f'CALL count_record_sales({album_id}, @sales_count)'
    print(f'Calling stored procedure: {call_stored_procedure}')
    session.execute(text(call_stored_procedure))
    result = session.execute(text('SELECT @sales_count;')).fetchall()[0][0]
    print(f'Number of sales for album 1 using the stored procedure: {result}')

Number of sales for album 1: 15
Calling stored procedure: CALL count_record_sales(1, @sales_count)
Number of sales for album 1 using the stored procedure: 15


### Procedure to get the total sales for all albums
```
DROP PROCEDURE IF EXISTS total_sales;
DELIMITER //
CREATE PROCEDURE total_record_sales(IN album_id INT, OUT total_sales NUMERIC)
BEGIN
	SET @album_id = album_id;
    SELECT 
        sum(sale_quantity * unit_sale_price) INTO total_sales
    FROM RECORD_SALES
    WHERE RECORD_SALES.album_id = @album_id;

END //
DELIMITER ;

In [4]:
import project.tools.db_utils as dbu
from sqlalchemy.sql import text

with dbu.get_session() as session:
    album_id = 1
    print(f'Total sales for album 1: {dbu.sqlalchemy_query_to_df("select sum(sale_quantity * unit_sale_price) as total_sales from record_sales where album_id = 1",)["total_sales"].values[0]}')
    call_stored_procedure = f'CALL total_record_sales({album_id}, @total_sales)'
    print(f'Calling stored procedure: {call_stored_procedure}')
    session.execute(text(call_stored_procedure))
    result = session.execute(text('SELECT @total_sales;')).fetchall()[0][0]
    print(f'Total sales for album 1 using the stored procedure: {result}')

Total sales for album 1: 150000000.0
Calling stored procedure: CALL total_record_sales(1, @total_sales)
Total sales for album 1 using the stored procedure: 150000000


## Function DDL

## Trigger DDL

## Normalization

## Integrity Enforcement

## Isolation Level

## Forms
- Table Updates
- Table Deletes
- Selects

### Table Inserts

In [6]:
from project.business_objects.record_genres import RecordGenres

test_genre_name = 'TEST Genre'
new_record = RecordGenres.create(genre_name=test_genre_name, genre_description='A new genre')
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
# Cleanup
RecordGenres.delete_by_name(test_genre_name)

,genre_name,genre_description
genre_id,,
83,TEST Genre,A new genre


### Table Updates

In [11]:
from project.business_objects.record_genres import RecordGenres

test_genre_name = 'TEST Genre'
updated_genre_name = 'Updated Genre'
new_record = RecordGenres.create(genre_name=test_genre_name, genre_description='A new genre')
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
new_record.genre_name = updated_genre_name
new_record.genre_description = 'An updated genre'
print(f'Record to update: {new_record}')
updated_record = RecordGenres.update(new_record)
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_name = '{updated_genre_name}'",  'genre_id'))
# Cleanup
RecordGenres.delete_by_name(updated_genre_name)

,genre_name,genre_description
genre_id,,
30,TEST Genre,A new genre


Record to update: RecordGenres(genre_id=30, genre_name='Updated Genre', genre_description='An updated genre')


,genre_name,genre_description
genre_id,,
30,Updated Genre,An updated genre


### Table Deletes

In [23]:
from project.business_objects.record_genres import RecordGenres

test_genre_name = 'TEST Genre'
new_record = RecordGenres.create(genre_name=test_genre_name, genre_description='A new genre')
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
RecordGenres.delete(new_record.genre_id)
print(f'After delete by id: {new_record.genre_id}')
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_id = '{new_record.genre_id}'",  'genre_id'))
new_record = RecordGenres.create(genre_name=test_genre_name, genre_description='A new genre')
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))
RecordGenres.delete_by_name(new_record.genre_name)
print(f'After delete by name: {new_record.genre_name}')
display(dbu.sqlalchemy_query_to_df(f"select * from record_genres where genre_name = '{test_genre_name}'",  'genre_id'))

,genre_name,genre_description
genre_id,,
35,TEST Genre,A new genre


After delete by id: 35


,genre_name,genre_description
genre_id,,


,genre_name,genre_description
genre_id,,
36,TEST Genre,A new genre


After delete by name: TEST Genre


,genre_name,genre_description
genre_id,,


### Table Reads

In [18]:
from project.business_objects.record_genres import RecordGenres
print(f'Read all genres')
for genre in RecordGenres.read_all()[:5]:
    print(genre)
    
print(f'Read by id: 5')
print(RecordGenres.read(5))
print(f'Read by name: Country')
print(RecordGenres.read_by_name('Country'))

Read all genres
RecordGenres(genre_id=1, genre_name='Rock', genre_description='Rock music')
RecordGenres(genre_id=2, genre_name='Pop', genre_description='Pop music')
RecordGenres(genre_id=3, genre_name='Rap', genre_description='Rap music')
RecordGenres(genre_id=4, genre_name='Country', genre_description='Country music')
RecordGenres(genre_id=5, genre_name='Jazz', genre_description='Jazz music')
Read by id: 5
RecordGenres(genre_id=5, genre_name='Jazz', genre_description='Jazz music')
Read by name: Country
[RecordGenres(genre_id=4, genre_name='Country', genre_description='Country music')]


## Reports